# Calculating sentiment of a piece of text 
## uses textblob API that internally uses Naive bayes classifier trained on Amazon review dataset

In [1]:
import nltk
import re
from os import path
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob

In [2]:
filename='single_review.txt'

####################################################################
file = open(filename, 'r')
text = file.read()
print(text)

ï»¿I was looking forward to trying this place for a long time.  The reviews on Yelp were good and I was told by a fellow steak lover that if I wanted a great (not the best) cut of steak at a reasonable price, then Texaz is the place to go.  I went on a Thursday evening and the place was pretty busy.  I was seated right away by a very friendly hostess.  You could tell from the looks of the place that it had been around a while.  I could have sworn my booth was going to fall apart, but it was ok.  The dÃƒÂ©cor is a bit eccentric with anything and everything having to do with Texas all over the walls.  All the patrons seemed to be having a good time and enjoying their meals.  There were also plenty of wait staff bustling around.


In [3]:
def parse_document(document):
    document = re.sub('\n', ' ', document)
    if isinstance(document, str):
        document = document
    elif isinstance(document, unicode):
        return unicodedata.normalize('NFKD', document).encode('ascii', 'ignore')
    else:
        raise ValueError('Document is not string or unicode!')
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

In [4]:
sentences=parse_document(text)

In [5]:
sentences = [nltk.word_tokenize(sentence) for sentence in sentences]

In [6]:
from textblob import TextBlob
#Get the sentiment of the body
count = 0
subjective_count=0
positive_count=0
negative_count=0
subjective_sentences=[]
positive_sentence=[]
negative_sentence=[]


In [7]:
for sentence in sentences:
    blob = TextBlob(str(sentence))
    sentiment = blob.sentiment
    count +=1
    if (sentiment.polarity>0 and sentiment.subjectivity >0 ):
        positive_sentence.append(sentence)
        positive_count +=1
    if (sentiment.polarity<0 and sentiment.subjectivity >0 ):
        negative_sentence.append(sentence)
        negative_count +=1
    
    if (sentiment.subjectivity >0):
        subjective_sentences.append(sentence)
        subjective_count +=1

obj_count=count-(subjective_count)

In [8]:
print ("subjective=", subjective_count)
print ("positive =", positive_count)
print ("negative =", negative_count)
print  ("count =",count )
print  ("objective sentences :",obj_count)


subjective= 7
positive = 5
negative = 1
count = 9
objective sentences : 2


##  So we can get some integer measure of the sentiment in the review text based on lexical semantics

In [9]:
if positive_count == negative_count: 
    print ("final sentiment = ",0 )

if positive_count > negative_count: 
    print ("final sentiment = ",positive_count-negative_count )
else:
    print ("final sentiment = ",negative_count-positive_count )

final sentiment =  4


## Emotion assessment using NRC lexicon 

Each line has the following format:
TargetWord<tab>AffectCategory<tab>AssociationFlag

TargetWord is a word for which emotion associations are provided.

AffectCategory is one of eight emotions (anger, fear, anticipation,
trust, surprise, sadness, joy, or disgust) or one of two polarities
(negative or positive).

AssociationFlag has one of two possible values: 0 or 1.  0 indicates
that the target word has no association with affect category,
whereas 1 indicates an association.


In [10]:
anger = 0
fear = 0
anticipation = 0
trust = 0
surprise = 0
sadness = 0
joy = 0
disgust = 0


In [11]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import sent_tokenize

file = open(filename, 'r')
textfromfile = file.read()
sentences=sent_tokenize(text)
sent = ''.join(sentences) # convert this to string


In [12]:
tok_words = TreebankWordTokenizer().tokenize(sent)
# Remove single-character tokens (mostly punctuation)
tok_words1 = [word for word in tok_words if len(word) > 1]
# Lowercase all words (default_stopwords are lowercase too)
tok_words2 = [word.lower() for word in tok_words1]


In [13]:
final_words = [word for word in tok_words2 if word not in stopwords.words('english')]
# Calculate frequency distribution
fdist = nltk.FreqDist(final_words)

In [14]:
import pandas as pd
nrc_lex = pd.read_csv( "NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt",sep='\t', names=['word','emotion','association'])
#nrc_lex.head()
print ("\n NRC Emotion lexicon loaded...")



 NRC Emotion lexicon loaded...


In [15]:
emotion_word=[]
for w1,w2 in fdist.items():
    if nrc_lex['word'].str.contains(w1).any():
        #print ("Found",w1)
        #print w1,w2
        #Change here ..every line is getting printed
        #print (nrc_lex.loc[nrc_lex['word'] == w1])
        anger_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='anger'].index.tolist()
        if len(anger_list) == 1:
            anger += w2*int(nrc_lex.iloc[int(anger_list[0])]['association'])
        fear_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='fear'].index.tolist()
        if len(fear_list) == 1:
            fear += w2*int(nrc_lex.iloc[int(fear_list[0])]['association'])
        anticipation_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='anticipation'].index.tolist()
        if len(anticipation_list) == 1:
            anticipation += w2*int(nrc_lex.iloc[int(anticipation_list[0])]['association'])
        trust_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='trust'].index.tolist()
        if len(trust_list) == 1:
            trust += w2*int(nrc_lex.iloc[int(trust_list[0])]['association'])
        surprise_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='surprise'].index.tolist()
        if len(surprise_list) == 1:
            surprise += w2*int(nrc_lex.iloc[int(surprise_list[0])]['association'])
        sadness_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='sadness'].index.tolist()
        if len(sadness_list) == 1:
            sadness += w2*int(nrc_lex.iloc[int(sadness_list[0])]['association'])
        joy_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='joy'].index.tolist()
        if len(joy_list) == 1:
            joy += w2*int(nrc_lex.iloc[int(joy_list[0])]['association'])
        disgust_list = nrc_lex[nrc_lex['word']==w1][nrc_lex['emotion']=='disgust'].index.tolist()
        if len(disgust_list) == 1:
            disgust += w2*int(nrc_lex.iloc[int(disgust_list[0])]['association'])
        print ("emotion word: ", w1)
        if w1 not in emotion_word:
            emotion_word.append(w1)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:26

emotion word:  forward
emotion word:  place
emotion word:  long
emotion word:  yelp
emotion word:  good
emotion word:  told
emotion word:  fellow
emotion word:  lover
emotion word:  great
emotion word:  best
emotion word:  cut
emotion word:  reasonable
emotion word:  price
emotion word:  go.i
emotion word:  went
emotion word:  evening
emotion word:  pretty
emotion word:  right
emotion word:  away
emotion word:  friendly
emotion word:  tell
emotion word:  booth
emotion word:  going
emotion word:  fall
emotion word:  apart
emotion word:  bit
emotion word:  eccentric
emotion word:  time
emotion word:  enjoying
emotion word:  plenty
emotion word:  wait
emotion word:  staff
emotion word:  bustling


##  So we can get the measure of the eight emotions in the review text 

In [16]:
print ("\n anger",anger)
print ("\n fear",fear)
print ("\n anticipation",anticipation)
print ("\n trust",trust)
print ("\n surprise",surprise)
print ("\n sadness",sadness)
print ("\n disgust", disgust)


 anger 1

 fear 1

 anticipation 9

 trust 7

 surprise 3

 sadness 1

 disgust 0


##  So we can get the list of emotion words in the review text.

In [17]:
print(len(emotion_word))
print(emotion_word)

33
['forward', 'place', 'long', 'yelp', 'good', 'told', 'fellow', 'lover', 'great', 'best', 'cut', 'reasonable', 'price', 'go.i', 'went', 'evening', 'pretty', 'right', 'away', 'friendly', 'tell', 'booth', 'going', 'fall', 'apart', 'bit', 'eccentric', 'time', 'enjoying', 'plenty', 'wait', 'staff', 'bustling']
